In [66]:
import numpy as np
import pandas as pd

from h3 import h3

import folium
from folium import plugins

from matplotlib import cm
import matplotlib
import matplotlib.pyplot as plt

import os
import time
from selenium import webdriver

import cv2

import pickle

In [67]:
# fname = 'data_hackathon_v04.csv'
fname = 'new_csv.csv'
df = pd.read_csv(fname, sep=',', index_col="ride_id", parse_dates=["created_at"])


In [68]:
# df_used = pd.read_csv("used_cells.csv", sep=',', index_col="cell_id")
# used_cell_ids = set(df_used.index.values)

In [42]:
locations_geo = df[["pickup_lat","pickup_lng"]].values

In [231]:
locations_grid = [h3.geo_to_h3(*loc, 8) for loc in locations_geo]
# hex_set = set(locations_grid)
hex_set = used_cell_ids

In [232]:
print(len(hex_set))

170


In [69]:
df["cell_id"] = df[["pickup_lat","pickup_lng"]].apply(lambda x: h3.geo_to_h3(*x, 7), axis=1)

In [70]:
df = df[df.cell_id.isin(hex_set)]

NameError: name 'hex_set' is not defined

In [ ]:
df.to_csv("with_cell_id.csv")

In [71]:
def limitTimeRange(df, start, end):
    sdf = df[(df.created_at >= start) & (df.created_at < end)]

    cell_visits_gbc = sdf.groupby("cell_id").count()
    cell_visits = dict(zip(cell_visits_gbc.index.values, cell_visits_gbc["created_at"].values))
#     print(len(cell_visits))

    return cell_visits, sdf

In [72]:
def convert_to_hex(rgba_color) :
    red = str(hex(rgba_color[0]))[2:].upper()
    green = str(hex(rgba_color[1]))[2:].upper()
    blue = str(hex(rgba_color[2]))[2:].upper()
    alpha = str(hex(rgba_color[3]))[2:].upper()

    if blue=='0':
        blue = '00'
    if red=='0':
        red = '00'
    if green=='0':
        green='00'
    if alpha=='0':
        alpha='00'

    return '#'+ red + green + blue + "FF" # + alpha

In [73]:
def plotMapFor(cell_visits, vmax = 500):
    gmap = folium.Map(location=[49.839922, 24.028741], tiles='Stamen Toner', zoom_start=13)

    # hm = plugins.HeatMap(sdf[["pickup_lat","pickup_lng"]].values)
    # print(hm)
    # hm
    # gmap.add_child(hm)
    # cells_boundaries

    # gj = folium.GeoJson(
    #     data={
    #         'type': 'Polygon',
    #         'coordinates': cells_boundaries
    #     })

    # gj.add_child(folium.Popup('outline Popup on GeoJSON'))
    # gj.add_to(gmap)
    # gmap

    cvv = cell_visits.values()
    norm = matplotlib.colors.Normalize(vmin=0, vmax=vmax)

    for cell_id, n_visits in cell_visits.items():
        poly = h3.h3_to_geo_boundary(cell_id)
        pgon = folium.Polygon(poly,
                    tooltip=str(n_visits),
                    fill_color=convert_to_hex(cm.jet(norm(n_visits), bytes=True)),
                    fill=True, fill_opacity=0.6,
                    stroke=False)
        pgon.add_to(gmap)
    
    return gmap

In [74]:
# %%time
cell_visits, sdf = limitTimeRange(df, "2018-08-18", "2019-02-19")
plotMapFor(cell_visits)

In [36]:
def cellsVisitsHourly(df, date):
    day = pd.Timedelta(days=1)
    hour = pd.Timedelta(hours=1)

    cell_ids = [dict() for i in range(24)]
    for i in range(7):
        for j in range(24):
            cvv = df[(df.created_at >= date+day*i+hour*j) & (df.created_at < date+day*i+hour*(j+1))].\
                        groupby(["cell_id"]).created_at.count()
            for k, v in zip(cvv.index.values, cvv.values): #df.created_at.dt.hour
                if k in cell_ids[j]:
                    cell_ids[j][k] += v / 7
                else:
                    cell_ids[j][k] = v / 7

    return cell_ids

# cellsVisitsHourly(df, pd.to_datetime("2019-01-10"))

In [37]:
def renderMapToImage(m, salt = 0):
    delay=2
    fn='testmap.html'
    tmpurl='file://{path}/{mapfile}'.format(path=os.getcwd(),mapfile=fn)
    m.save(fn)

    browser = webdriver.Chrome()
    browser.get(tmpurl)

    time.sleep(delay)
    
    imname = "map_{}.png".format(salt)
    browser.save_screenshot(imname)
    browser.quit()
    
    return cv2.imread(imname)

# img = renderMapToImage(plotMapFor(cell_visits))
# plt.imshow(img)

In [38]:
week = pd.Timedelta(weeks=1)

# for i in range(4):
i = 0
visits_h = cellsVisitsHourly(df, pd.to_datetime("2018-12-01") + week*i)

for j in range(24):
    img = renderMapToImage(plotMapFor(visits_h[j], vmax=20), salt=("{}-{:02}".format(i, j)))

#     print(img)
#     plt.imshow(img)

WebDriverException: Message: 'chromedriver' executable needs to be in PATH. Please see https://sites.google.com/a/chromium.org/chromedriver/home


In [ ]:
subsample = df[(df.created_at > "2019-01-03") & (df.created_at < "2019-01-04")][
    ["pickup_lat","pickup_lng","dropoff_lat","dropoff_lng"]].values[:10]


In [239]:
start_date = pd.to_datetime("2018-08-21")
hour = pd.Timedelta(hours=1)

dataset = []
n_zones = len(hex_set)
id_lookup = dict(zip(sorted(list(hex_set)), range(len(hex_set))))
for i in range(24*170):
    data_points = [0] * n_zones
    current_cell_visits, sdf = limitTimeRange(df, start_date, start_date + hour)
    start_date += hour
    for k, v in current_cell_visits.items():
        data_points[id_lookup[k]] += v
    
    dataset.append(data_points)

In [241]:
img = np.array(dataset)
with open("dataset.pickle", "wb") as f:
    pickle.dump(img, f)

cv2.imwrite("/tmp/kek.png", img)

True